# Kinematic Trajectory Optimization
## Ball throwing and catching

In [8]:
import time

import numpy as np
from pydrake.all import (AddMultibodyPlantSceneGraph, BsplineTrajectory,
                         DiagramBuilder, KinematicTrajectoryOptimization,
                         MeshcatVisualizer, MeshcatVisualizerParams,
                         MinimumDistanceConstraint, Parser, PositionConstraint,
                         Rgba, RigidTransform, Role, Solve, Sphere,
                         StartMeshcat, JacobianWrtVariable, RollPitchYaw)

from manipulation import running_as_notebook
from manipulation.meshcat_utils import (PublishPositionTrajectory,
                                        model_inspector)
from manipulation.scenarios import AddIiwa, AddPlanarIiwa, AddShape, AddWsg
from manipulation.utils import AddPackagePaths, FindResource


In [9]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7001


In [17]:
X_WStart = RigidTransform([0.5, 0, 0.15])
X_WGoal = RigidTransform(RollPitchYaw([-0.16, 3.14, -1.92]),[0, -0.6, 0.15])
q_start = np.array([-0.54617387, 1.13925077, 0.13639149, -1.50426796, 1.0835878, 1.701366, 0.50004928])
q_goal = np.array([-2.00188489, 1.23067234, 0.03573024, -1.42203902, 1.57121904, 1.71564864, -2.64577454])
V_Start_translational = np.array([-0.55339859, -0.66407831,  4.42718872])
V_Goal_translational = V_Start_translational * [1,1,-1]
t_duration = 0.9035079029052512

In [19]:
import time

import numpy as np
from IPython.display import clear_output
from pydrake.all import (AddMultibodyPlantSceneGraph, DiagramBuilder,
                         JacobianWrtVariable, MathematicalProgram,
                         MeshcatVisualizer, PiecewisePolynomial, Solve,
                         StartMeshcat)

from manipulation import running_as_notebook
from manipulation.meshcat_utils import plot_mathematical_program


def SpatialVelToJointVelConverter():
    builder = DiagramBuilder()

    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
    iiwa = AddIiwa(plant)
    wsg = AddWsg(plant, iiwa, welded=True, sphere=True)
    gripper_frame = plant.GetFrameByName("body", wsg)
    plant.Finalize()

    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(context)

    def convert_spatial_vel_to_joint_vel(q, v_Gdesired):
        plant.SetPositions(plant_context, q)
        diagram.Publish(context)

        clear_output(wait=True)
        J_G = plant.CalcJacobianTranslationalVelocity(plant_context,
                                                      JacobianWrtVariable.kQDot,
                                                      gripper_frame, [0, 0, 0],
                                                      plant.world_frame(),
                                                      plant.world_frame())
        print("J_G = ")
        print(
            np.array2string(J_G,
                            formatter={'float': lambda x: "{:5.2f}".format(x)}))
        print(np.shape(J_G))
        print(np.shape(v_Gdesired))
        # print(J_G @ v_Gdesired.T)

    return convert_spatial_vel_to_joint_vel

converter = SpatialVelToJointVelConverter()

In [20]:
converter(q_start, V_Start_translational)

J_G = 
[[ 0.07 -0.15  0.11  0.32  0.15 -0.08 -0.00]
 [ 0.50  0.09  0.35 -0.19  0.00  0.00  0.00]
 [ 0.00 -0.46  0.18  0.11  0.08  0.15  0.00]]


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 3 is different from 7)

In [14]:
converter(q_goal,X_WGoal)

J_G = 
[[ 0.58  0.08  0.36 -0.14 -0.06  0.01 -0.00]
 [-0.06  0.17 -0.10 -0.30 -0.13  0.08  0.00]
 [ 0.00 -0.55  0.17  0.18  0.08  0.15  0.00]]


In [5]:
def trajopt_pv_pair_demo():
    meshcat.Delete()
    builder = DiagramBuilder()

    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)

    parser = Parser(plant)
    AddPackagePaths(parser)
    iiwa = AddIiwa(plant)
    wsg = AddWsg(plant, iiwa, welded=True, sphere=True)
    X_WStart = RigidTransform([0.5, 0, 0.15])
    meshcat.SetObject("start", Sphere(0.02), rgba=Rgba(.9, .1, .1, 1))
    meshcat.SetTransform("start", X_WStart)
    X_WGoal = RigidTransform([0, -0.6, 0.15])
    meshcat.SetObject("goal", Sphere(0.02), rgba=Rgba(.1, .9, .1, 1))
    meshcat.SetTransform("goal", X_WGoal)

    t_start = 0.5
    num_control_points = 11

    plant.Finalize()

    visualizer = MeshcatVisualizer.AddToBuilder(
        builder, scene_graph, meshcat,
        MeshcatVisualizerParams(role=Role.kIllustration))
    collision_visualizer = MeshcatVisualizer.AddToBuilder(
        builder, scene_graph, meshcat,
        MeshcatVisualizerParams(prefix="collision", role=Role.kProximity))
    meshcat.SetProperty("collision", "visible", False)

    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(context)

    q0 = plant.GetPositions(plant_context)
    gripper_frame = plant.GetFrameByName("body", wsg)
    trajopt = KinematicTrajectoryOptimization(plant.num_positions(), num_control_points)
    prog = trajopt.get_mutable_prog()
    
    trajopt.AddPathLengthCost(1.0)
    # Add cost for acceleration?

    trajopt.AddPositionBounds(plant.GetPositionLowerLimits(),
                              plant.GetPositionUpperLimits())
    trajopt.AddVelocityBounds(plant.GetVelocityLowerLimits(),
                              plant.GetVelocityUpperLimits())

    trajopt.AddDurationBounds(t_start, t_start + 2 * t_duration) # Assume t_duration each way
    trajopt.AddEqualTimeIntervalConstraints()

    # start constraint
    start_constraint = PositionConstraint(plant, plant.world_frame(),
                                          X_WStart.translation(),
                                          X_WStart.translation(), gripper_frame,
                                          [0, 0.1, 0], plant_context)
    trajopt.AddPathConstraint(start_constraint, 0)
    prog.AddQuadraticErrorCost(np.eye(7), q0,
                               trajopt.path_control_points()[:, 0])

    # goal constraint
    goal_constraint = PositionConstraint(plant, plant.world_frame(),
                                         X_WGoal.translation(),
                                         X_WGoal.translation(), gripper_frame,
                                         [0, 0.1, 0], plant_context)
    
    trajopt.AddPathConstraint(goal_constraint, 0.5)
    prog.AddQuadraticErrorCost(np.eye(7), q0,
                               trajopt.path_control_points()[:, np.round(num_control_points/2).astype(int)])

    # return to start
    trajopt.AddPathConstraint(start_constraint, 1)
    prog.AddQuadraticErrorCost(np.eye(7), q0,
                               trajopt.path_control_points()[:, -1])

    
    # start and end with zero velocity
    start_V = np.zeros((7,1))
    end_V = np.zeros((7,1))
    
    # end_V[6]=-0.3
    trajopt.AddFixedPathVelocityConstraint(start_V, 0)
    trajopt.AddFixedPathVelocityConstraint(end_V, 0.5)
    trajopt.AddFixedPathVelocityConstraint(start_V, 1)

    # Solve once without the collisions and set that as the initial guess for
    # the version with collisions.
    result = Solve(prog)
    if not result.is_success():
        print("Trajectory optimization failed, even without collisions!")
        print(result.get_solver_id().name())
    trajopt.SetPathInitialGuess(trajopt.ReconstructPath(result))

    PublishPositionTrajectory(trajopt.ReconstructPath(result), context, plant,
                              visualizer)
    collision_visualizer.Publish(
        collision_visualizer.GetMyContextFromRoot(context))

    return
    # collision constraints
    collision_constraint = MinimumDistanceConstraint(plant, 0.001,
                                                     plant_context, None, 0.01)
    evaluate_at_s = np.linspace(0, 1, 25)
    for s in evaluate_at_s:
        trajopt.AddPathConstraint(collision_constraint, s)

    result = Solve(prog)
    if not result.is_success():
        print("Trajectory optimization failed")
        print(result.get_solver_id().name())

    PublishPositionTrajectory(trajopt.ReconstructPath(result), context, plant,
                              visualizer)
    collision_visualizer.Publish(
        collision_visualizer.GetMyContextFromRoot(context))
    
    

trajopt_pv_pair_demo()
